In [2]:
from sklearn.neighbors import KNeighborsClassifier

# sklearn metrics
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split


# images
from scipy.ndimage import convolve
from skimage.feature import hog
from skimage import draw, data, io, segmentation, color, exposure
from skimage.measure import regionprops
from skimage.filters import threshold_otsu
from skimage.transform import resize 
from skimage.transform import warp 
from PIL import Image

# basics
import pandas as pd
import numpy as np
from pprint import pprint
from PIL import Image
from pytesseract import image_to_string



ImportError: No module named 'pytesseract'

In [3]:
import math

def img_round(x, base=75):
    """
    Now useless function (replaced by binarization) for flattening image data
    """
    return (base * math.floor(float(x)/base))
vround = np.vectorize(img_round) 

def get_img(i, size):
    """
    Returns a binary image from my file directory with index i
    """
    img = Image.open('/home/yasho/OCR files/train/'+ str(i+1) + '.Bmp')
    img = img.convert("L")
    img = img.resize((size,size))
    image = np.asarray(img)
    image.setflags(write=True)
    thresh = threshold_otsu(image)
    binary = image > thresh
    return binary

def nudge_dataset(X, Y, size):
    """
    This produces a dataset 5 times bigger than the original one,
    by moving the (size x size) images around by 1px to left, right, down, up
    """
    direction_vectors = [
        [[0, 1, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [1, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 1],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 1, 0]]]

    shift = lambda x, w: convolve(x.reshape((size, size)), mode='constant',
                                  weights=w).ravel()
    X = np.concatenate([X] +
                       [np.apply_along_axis(shift, 1, X, vector)
                        for vector in direction_vectors])
    Y = np.concatenate([Y for _ in range(5)], axis=0)
    return X, Y

def show_img(img):
    width = 5.0
    height = img.shape[0]*width/img.shape[1]
    f = plt.figure(figsize=(width, height))
    plt.imshow(img)

In [4]:


df = pd.read_csv('/home/yasho/OCR files/trainLabels.csv', header=0)
raw_y = np.asarray(df['Class'])
raw_x = np.asarray([get_img(i, 50) for i in df.index]).astype(float)
x = np.asarray([i.ravel() for i in raw_x])
y = raw_y
print (x.shape, y.shape)



(6283, 2500) (6283,)


In [6]:
knn = KNeighborsClassifier(n_neighbors=5)

# fit
knn.fit(X_train, Y_train)
print('RJ')

RJ


In [5]:
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y)

#nudge dataset improves performance: test and see!
X_train, Y_train = nudge_dataset(X_train, Y_train, 50)


In [7]:
def get_test_img(i):
    """
    Returns image from my file directory with corresponding index i
    """
    img = Image.open('/home/yasho/OCR files/test/'+ str(i) + '.Bmp')
    img = img.convert("L")
    img = img.resize((50,50))
    image = np.asarray(img)
    image.setflags(write=True)
    thresh = threshold_otsu(image)
    binary = image > thresh
    return binary

In [9]:
from random import randint

random_image = 7062
print(random_image)
print("My Guess for this file is:" + str(knn.predict(get_test_img(random_image).ravel())).strip('[]\''))
#show_img(get_test_img(random_image))

7062
My Guess for this file is:T


/home/yasho/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [10]:
import pytesseract
from gtts import gTTS
import os

from PIL import Image, ImageEnhance, ImageFilter

im = Image.open("/home/yasho/dbk.png") # the second one 
im = im.convert('RGB')
im = im.filter(ImageFilter.MedianFilter())
enhancer = ImageEnhance.Contrast(im)
im = enhancer.enhance(2)
im = im.convert('1')
im.save('/home/yasho/temp2.jpg')
text1 = pytesseract.image_to_string(Image.open('/home/yasho/dbk.png'))
text_file = open("/home/yasho/Output.txt", "w")
text_file.write(text1)
text_file.close()
tts = gTTS(text=text1, lang='en')
tts.save("/home/yasho/out.mp3")
os.system("mpg321 /home/yasho/out.mp3")
print(text1)